In [2]:
import spacy

sp = spacy.load("ru_core_news_lg")

In [3]:
import pandas as pd
from docx import Document
import os


def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for paragraph in doc.paragraphs:
        full_text.append(paragraph.text)
    return "\n".join(full_text)


def load_docs(dataset_path):
    df = pd.DataFrame(columns=["doc", "text"])
    for file_path in os.listdir(dataset_path):
        if file_path.startswith("~$"):
            continue
        text = read_docx(dataset_path + file_path)
        df.loc[len(df.index)] = [file_path, text]
    return df


df = load_docs("data/text/")
df["type"] = df.apply(lambda row: 0 if str(row["doc"]).startswith("tz_") else 1, axis=1)
df.info()
df.sort_values(by=["doc"], inplace=True)

display(df.head(), df.tail())

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, 0 to 40
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   doc     41 non-null     object
 1   text    41 non-null     object
 2   type    41 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.3+ KB


,doc,text,type
16,tz_01.docx,2.2 Техническое задание\n2.2.1 Общие сведения\...,0
19,tz_02.docx,2.2 Техническое задание\n2.2.1 Общие сведения\...,0
28,tz_03.docx,2.2. Техническое задание\nОбщие сведения:\nВ д...,0
35,tz_04.docx,Техническое задание\n2.2.1 Общие сведения\nИнт...,0
38,tz_05.docx,2.2 Техническое задание\n2.2.1 Общие сведения....,0


,doc,text,type
25,Этапы разработки проекта2.docx,Этапы разработки проекта: заключительные стади...,1
21,Этапы разработки проекта3.docx,Этапы разработки проекта: определение стратеги...,1
40,Этапы разработки проекта4.docx,"Этапы разработки проекта: реализация, тестиров...",1
30,Этапы разработки проекта5.docx,Этапы разработки проекта: стратегия и анализ\n...,1
22,Язык манипуляции данными.docx,2.1.3. Язык манипуляции данными (ЯМД)\nЯзык ма...,1


In [32]:
from gensim.models.phrases import Phraser, Phrases

def prep_text(text):
    doc = sp(text)
    lower_sents = []
    for sent in doc.sents:
        lower_sents.append([word.lemma_.lower() for word in sent if not word.is_punct and not word.is_stop and not word.is_space])
    lower_bigram = Phraser(Phrases(lower_sents))
    clean_sents = []
    for sent in lower_sents:
        clean_sents.append(lower_bigram[sent])
    return clean_sents

df["prep_text"] = df.apply(lambda row: prep_text(row["text"]), axis=1)
df

,doc,text,type,prep_text
16,tz_01.docx,2.2 Техническое задание\n2.2.1 Общие сведения\...,0,"[[2.2, технический, задание, 2.2.1, общий, све..."
19,tz_02.docx,2.2 Техническое задание\n2.2.1 Общие сведения\...,0,"[[2.2, технический, задание, 2.2.1, общий, све..."
28,tz_03.docx,2.2. Техническое задание\nОбщие сведения:\nВ д...,0,"[[2.2], [технический, задание, общий, сведение..."
35,tz_04.docx,Техническое задание\n2.2.1 Общие сведения\nИнт...,0,"[[технический, задание, 2.2.1, общий, сведение..."
38,tz_05.docx,2.2 Техническое задание\n2.2.1 Общие сведения....,0,"[[2.2, технический, задание, 2.2.1, общий, све..."
2,tz_06.docx,2.2 Техническое задание\t\n1.Общие сведения\nП...,0,"[[2.2, технический, задание, 1.общие, сведение..."
4,tz_07.docx,Техническое задание\nОбщие сведения\nВ данном ...,0,"[[технический, задание, общий, сведение, разде..."
33,tz_08.docx,Техническое задание\n1 Общие сведения\n1.1 Пол...,0,"[[технический, задание, 1, общий, сведение, 1...."
39,tz_09.docx,2.2. Техническое задание\n2.2.1.\n\nОбщие свед...,0,"[[2.2], [технический, задание, 2.2.1, общий, с..."
1,tz_10.docx,2.2. Техническое задание\n2.2.1. Общие сведени...,0,"[[2.2], [технический, задание, 2.2.1], [общий,..."


In [33]:
from gensim.models.word2vec import Word2Vec

word2vec = Word2Vec(
    sentences=df["prep_text"].explode().tolist(),
    vector_size=64,
    sg=1,
    window=10,
    epochs=5,
    min_count=10,
    workers=4,
    seed=9,
)

In [34]:
word2vec.wv.key_to_index

{'система': 0,
 'работа': 1,
 'требование': 2,
 'база': 3,
 'пользователь': 4,
 'разработка': 5,
 'модель': 6,
 'информация': 7,
 'субд': 8,
 'этап': 9,
 'ошибка': 10,
 'функция': 11,
 'являться': 12,
 'таблица': 13,
 'средство': 14,
 'проект': 15,
 'сервер': 16,
 'процесс': 17,
 'документ': 18,
 'программа': 19,
 'использовать': 20,
 'состояние': 21,
 'проектирование': 22,
 'программный': 23,
 'случай': 24,
 'создание': 25,
 'модуль': 26,
 'время': 27,
 'объект': 28,
 'заказчик': 29,
 'приложение': 30,
 'реализация': 31,
 'правило': 32,
 'результат': 33,
 'вид': 34,
 'разработчик': 35,
 'возможность': 36,
 'технический': 37,
 'метод': 38,
 'тестирование': 39,
 'следующий': 40,
 'задача': 41,
 'использование': 42,
 'обработка': 43,
 'часть': 44,
 'позволять': 45,
 'клиент': 46,
 'интерфейс': 47,
 'переход': 48,
 'выполнение': 49,
 'обеспечение': 50,
 'информационный_система': 51,
 'новый': 52,
 'изменение': 53,
 'атрибут': 54,
 'основный': 55,
 'код': 56,
 'решение': 57,
 'запрос': 58,

In [40]:
word2vec.wv.most_similar("1с", topn=3)

[('8.1', 0.9820374250411987),
 ('бухгалтерия', 0.9796695709228516),
 ('конфигурация', 0.9309621453285217)]

In [41]:
word2vec.wv["1с"]

array([-0.6908452 , -0.28973973, -0.13875388, -0.71968764, -0.2846077 ,
       -0.03394265, -0.11022742,  0.00537843,  0.25884008, -0.07338168,
        0.51379067,  0.14176723,  0.23107368, -0.23853333, -0.08213347,
        0.22784428,  0.0082625 ,  0.08487764, -0.45476234,  0.31248948,
       -0.11175565,  0.02592653,  0.35858843,  0.30309716, -0.03435089,
       -0.00924653,  0.757564  ,  0.3493382 , -0.26192346, -0.04177783,
        0.29669356, -0.10495648, -0.0755821 , -0.18037044, -0.00430617,
        0.77395517,  0.19479573,  0.5982179 ,  0.00795395, -0.01108605,
       -0.05196482,  0.3891119 ,  0.35530448, -0.15757816,  0.02234032,
        0.37822202, -0.53775096, -0.10007993, -0.76245916, -0.4506751 ,
       -0.5795265 ,  0.21962252,  0.29487562,  0.06301279,  0.2006733 ,
        0.5975451 , -0.03358367,  0.803588  ,  0.36706072, -0.645835  ,
        0.21826494, -0.10143226,  0.5875708 , -0.3040449 ], dtype=float32)

In [42]:
word2vec.wv.doesnt_match("java php 1с oracle заказчик".split())

'заказчик'

In [43]:
word2vec.wv.similarity("java", "javascript")

0.850643

In [44]:
word2vec.wv.similarity("java", "1с")

0.72610414

In [53]:
word2vec.wv.most_similar(positive=["1с", "машина"], negative=["бухгалтерия"])

[('java', 0.9692181944847107),
 ('лингвистический', 0.9653875827789307),
 ('php', 0.9624669551849365),
 ('mysql', 0.9328854084014893),
 ('аппаратный', 0.9266995191574097),
 ('работать', 0.918877124786377),
 ('знание', 0.9152171611785889),
 ('apache', 0.9150222539901733),
 ('microsoft', 0.9149153828620911),
 ('клиентский', 0.914190948009491)]

In [59]:
word2vec.wv.most_similar(positive=["субд", "язык"], negative=["база"])

[('программирование', 0.8054100871086121),
 ('java', 0.7383090853691101),
 ('использовать', 0.7284875512123108),
 ('использование', 0.7234159708023071),
 ('позволять', 0.7068267464637756),
 ('sql', 0.7053561806678772),
 ('php', 0.7042484879493713),
 ('моделирование', 0.6964462399482727),
 ('работать', 0.6867913603782654),
 ('стандарт', 0.6866309642791748)]

In [55]:
word2vec.wv.most_similar(positive=["java", "бухгалтерия"], negative=["машина"])

[('1с', 0.9693869352340698),
 ('8.1', 0.955109179019928),
 ('среда', 0.9511857628822327),
 ('ориентировать', 0.9304741024971008),
 ('навык', 0.9272575974464417),
 ('web', 0.9159443378448486),
 ('конфигурация', 0.91340172290802),
 ('специалист', 0.9114348888397217),
 ('лингвистический', 0.906047523021698),
 ('рабочий', 0.9038367867469788)]

In [64]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, max_iter=1000)
coords_df = pd.DataFrame(
    tsne.fit_transform(word2vec.wv[word2vec.wv.key_to_index]), columns=["x", "y"]
)
coords_df["token"] = word2vec.wv.key_to_index.keys()
coords_df

,x,y,token
0,-23.278934,13.247012,система
1,-26.849134,23.807318,работа
2,-24.966194,8.754627,требование
3,-15.748887,-24.560507,база
4,-14.334096,-9.731017,пользователь
...,...,...,...
1038,17.863838,2.956259,некий
1039,6.895282,-10.811954,самым
1040,10.354476,1.916842,похожий
1041,7.145860,-2.575255,целесообразный


In [ ]:
from bokeh.io import output_notebook
from bokeh.plotting import show, figure

output_notebook()
p = figure(width=800, height=800)
p.text(x=coords_df.x, y=coords_df.y, text=coords_df.token) # type: ignore
show(p)

Loading BokehJS ...